Load file and remove background signal

In [ ]:
from pathlib import Path

import hbn_pl.io as io
import hbn_pl.preprocess as preprocess
import hbn_pl.plot as plot
import hbn_pl.peaks as peaks
import hbn_pl.energy as energy

DATA_DIR = 'C:\\Users\\emmah\\OneDrive - University of Cambridge\\Data for IBS November 2025\\Sample 5\\PL spectra'
DATA_DIR = Path(DATA_DIR)
FILE_NAME = '2025-11-17 16_29_50 sample5_DCMd2_200uw_emitter6 2184.spe'
#FILE_NAME = '2025-11-17 17_34_30 sample5_DCMd2_200uw_emitter11 2189.spe'
#FILE_NAME = '2025-11-17 17_59_53 sample5_DCMd2_200uw_emitter17 2195.spe'
spe_path = DATA_DIR / FILE_NAME
wavelength, frames = io.load_spe(spe_path)
frames = frames[0:15]
frames = preprocess.background_subtract(frames, bg_slice=(1, 50))
plot.plot_frames(wavelength, frames)

Cosmic ray removal

In [ ]:
frames_norm = preprocess.normalise(frames)
cosmic_frames = preprocess.detect_cosmic_frames(frames_norm, sigma_threshold=90, min_outliers=3)
frames = preprocess.remove_cosmic_rays(frames, cosmic_frames, sigma=2.5)

Bad frame removal

In [ ]:
drop_fraction = 0.6
bad_frames = preprocess.detect_bad_frames_simple(frames, drop_fraction = drop_fraction)
#bad_frames = preprocess.detect_bad_frames_complex(frames, window = 5, drop_fraction = drop_fraction)
#frames = preprocess.remove_frames(frames, bad_frames)

Average and normalise

In [ ]:
avg, avg_norm = preprocess.average_and_normalise(frames)
plot.plot_spectrum(wavelength, avg_norm)

Finished for now 